In [6]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func, timer
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
from joblib import Parallel, delayed
logger = logger_func()

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr_torch.models import *
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_fixlen_feature_names
import torch

2019-09-23 14:36:51,028 func.utils 347 [INFO]    [logger_func] start 
2019-09-23 14:36:51,028 func.utils 347 [INFO]    [logger_func] start 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMN_GROUP = 'DT-M'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, COLUMN_GROUP, 'is_train', 'date']

def filter_feature(path):
    if path.count(''):
        return True
    else:
        return False

# paths_train = glob('../feature/raw_use/*_train.gz')
# paths_test  = glob('../feature/raw_use/*_test.gz')
paths_train = glob('../submit/re_sub/*_train.gz')
paths_test  = glob('../submit/re_sub/*_test.gz')
paths_train += glob('../submit/add_feature/*_train.gz')
paths_test  += glob('../submit/add_feature/*_test.gz')
paths_train += glob('../feature/valid_use/531*_train.gz')
paths_test  += glob('../feature/valid_use/531*_test.gz')
paths_train += glob('../feature/valid_use/532*_train.gz')
paths_test  += glob('../feature/valid_use/532*_test.gz')
# paths_train += glob('../feature/valid_trush/532*uid3*_train.gz')
# paths_test  += glob('../feature/valid_trush/532*uid3*_test.gz')

paths_train_feature = []
paths_test_feature  = []

df_train = parallel_load_data(paths_train)
df_test  = parallel_load_data(paths_test)
Y = df_train[COLUMN_TARGET]
df_train.drop(COLUMN_TARGET, axis=1, inplace=True)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)

In [24]:
sparse_features = []
dense_features = [col for col in data.columns if col not in COLUMNS_IGNORE]

data[sparse_features] = data[sparse_features].fillna('-1', )


def parallel_fillna(feature):
    avg = feature.mean()
    feature.fillna(avg, inplace=True)
    return feature


list_df = []
with timer("Fill Na"):
    for col in tqdm(dense_features):
        avg = data[col].mean()
        data[col].fillna(avg, inplace=True)

SystemExit: 

In [16]:
target = [COLUMN_TARGET]

# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
    
    
def parallel_mms(feature, col):
    return feature

dense_features = [col for col in dense_features if not col.count('513__D2-D4__ratio__ProductCD-W')]
    
with timer("Min Max Scalar"):
    for col in tqdm(dense_features):
        mms = MinMaxScaler(feature_range=(0, 1))
        data[col] = mms.fit_transform(data[col].values.reshape(data.shape[0], -1))



  0%|          | 0/1043 [00:00<?, ?it/s]

  0%|          | 2/1043 [00:00<01:01, 17.06it/s]

  0%|          | 4/1043 [00:00<01:08, 15.27it/s]

  1%|          | 6/1043 [00:00<01:13, 14.12it/s]

  1%|          | 8/1043 [00:00<01:17, 13.33it/s]

  1%|          | 10/1043 [00:00<01:25, 12.15it/s]

  1%|          | 12/1043 [00:00<01:26, 11.96it/s]

  1%|▏         | 14/1043 [00:01<01:27, 11.81it/s]

  2%|▏         | 16/1043 [00:01<01:33, 10.94it/s]

  2%|▏         | 18/1043 [00:01<01:37, 10.52it/s]

  2%|▏         | 20/1043 [00:01<01:38, 10.40it/s]

  2%|▏         | 22/1043 [00:01<01:39, 10.25it/s]

  2%|▏         | 24/1043 [00:02<01:35, 10.72it/s]

  2%|▏         | 26/1043 [00:02<01:31, 11.07it/s]

  3%|▎         | 28/1043 [00:02<01:33, 10.87it/s]

  3%|▎         | 30/1043 [00:02<01:30, 11.21it/s]

  3%|▎         | 32/1043 [00:02<01:30, 11.14it/s]

  3%|▎         | 34/1043 [00:03<01:32, 10.95it/s]

  3%|▎         | 36/1043 [00:03<01:34, 10.69it/s]

  4%|▎         | 38/1043 [00:03<01:32, 10.

[Min Max Scalar] done in 99 s


In [34]:
dense_features = [col for col in dense_features if not col.count('513__D2-D4__ratio__ProductCD-W')]

'513__D2-D4__ratio__ProductCD-W' in dense_features

False

In [35]:
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                          for feat in sparse_features] + [DenseFeat(feat, 1,)
                                                          for feat in dense_features]

dnn_feature_columns    = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

fixlen_feature_names = get_fixlen_feature_names(
    linear_feature_columns + dnn_feature_columns)

In [36]:
for col in dense_features:
    if col.count('D2-D4'):
        print(col)
#     print(df_train[col].max(), df_train[col].min(), col)
#     num = df_train[col].isnull().sum()
#     if num>0:
#         print(col)
# for c in use_cols:
#     if c.count('Fraud'):
#         print(c)

513__D2-D4__diff__ProductCD-W


In [18]:
from sklearn.model_selection import GroupKFold

df_train = data.iloc[:len(df_train)]
df_test  = data.iloc[len(df_train):]

n_splits = 6
group_kfold_path = '../input/0908_ieee__DT-M_GroupKFold.gz'
group = read_pkl_gzip(group_kfold_path)
df_train[COLUMN_GROUP] = group

kfold = list(GroupKFold(n_splits=n_splits).split(df_train, Y, df_train[COLUMN_GROUP]))
print("Set Kfold")

Set Kfold


[(array([134339, 134340, 134341, ..., 590537, 590538, 590539]),
  array([     0,      1,      2, ..., 134336, 134337, 134338])),
 (array([     0,      1,      2, ..., 590537, 590538, 590539]),
  array([312574, 312575, 312576, ..., 414539, 414540, 414541])),
 (array([     0,      1,      2, ..., 590537, 590538, 590539]),
  array([134339, 134340, 134341, ..., 226846, 226847, 226848])),
 (array([     0,      1,      2, ..., 498110, 498111, 498112]),
  array([498113, 498114, 498115, ..., 590537, 590538, 590539])),
 (array([     0,      1,      2, ..., 590537, 590538, 590539]),
  array([226849, 226850, 226851, ..., 312571, 312572, 312573])),
 (array([     0,      1,      2, ..., 590537, 590538, 590539]),
  array([414542, 414543, 414544, ..., 498110, 498111, 498112]))]

In [77]:
# 3.generate input data for model

use_cols = dense_features

oof_pred = np.zeros(df_train.shape[0])
x_test = df_test[use_cols]
test_preds = []
test_model_input = [x_test[name] for name in fixlen_feature_names]

for n_fold, (trn_idx, val_idx) in enumerate(kfold):
    
#     if n_fold <5:
#         continue
    
    with timer("Preset Dataset."):
        x_train = df_train.iloc[trn_idx][use_cols]
        y_train = Y.iloc[trn_idx]
        x_valid = df_train.iloc[val_idx][use_cols]
        y_valid = Y.iloc[val_idx]

        train_model_input = [x_train[name] for name in fixlen_feature_names]
        valid_model_input = [x_valid[name] for name in fixlen_feature_names]

    # 4.Define Model,train,predict and evaluate
    
    print(f"Fold{n_fold} Start Train and Predict")
    with timer("Fitting"):

        device = 'cpu'
        use_cuda = True
        if use_cuda and torch.cuda.is_available():
            print('cuda ready...')
            device = 'cuda:0'

        model = xDeepFM(
            linear_feature_columns=linear_feature_columns,
            dnn_feature_columns=dnn_feature_columns,
            task='binary',
            l2_reg_embedding=1e-5,
            device=device
        )

        model.compile(
            "adagrad",
            "binary_crossentropy",
            metrics=["binary_crossentropy", "auc"],
        )
        model.fit(
            train_model_input,
            y_train.values,
            batch_size=2048,
            epochs=7,
            validation_split=0.1,
            verbose=2
        )

    with timer("Predict"):
        
        pred_ans = model.predict(valid_model_input, 256)
        print("")
        print("test LogLoss", round(log_loss(y_valid, pred_ans), 4))
        print("test AUC", round(roc_auc_score(y_valid.values, pred_ans), 4))
        
        oof_pred[val_idx] = pred_ans.ravel()
        
        test_pred = model.predict(test_model_input, 256)
        test_preds.append(test_pred)

[Preset Dataset.] done in 6 s
Fold0 Start Train and Predict


IndexError: index 0 is out of bounds for dimension 0 with size 0

In [68]:
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())[:13]

# test_pred = np.mean(test_preds, axis=0).ravel()
base_train = read_pkl_gzip('../input/base_train.gz')
base_test = read_pkl_gzip('../input/base_test.gz')
base = pd.concat([base_train, base_test], axis=0)
result = pd.DataFrame(np.vstack([base[COLUMN_ID].values, np.hstack((oof_pred, test_pred))]).T, columns=[COLUMN_ID, 'pred_xdeepfm'])
result[COLUMN_ID] = result[COLUMN_ID].astype('int')
result.to_csv(f'../output/pred_result/{start_time[:13]}_ieee__xdeepfm_pred_{len(use_cols)}feature_AUC0-870')